In [29]:
import torch
import torch.nn as nn

In [30]:
class DropoutFromScratch(nn.Module):
    
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p 
        
    def forward(self, x):
        if self.training:
            mask = (torch.rand_like(x) > self.p).float()
            return x * mask / (1 - self.p)
        else:
            return x

class BatchNorm1dFromScratch(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super().__init__()
        self.eps = eps
        self.momentum = momentum
        
        self.gamma = nn.Parameter(torch.ones(num_features))
        self.beta = nn.Parameter(torch.zeros(num_features))
        
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))
        
    def forward(self, x):
        if self.training:
            batch_mean = x.mean(dim=0)
            batch_var = x.var(dim=0, unbiased=False)
        
            # For running stats, use unbiased variance
            batch_var_unbiased = x.var(dim=0, unbiased=True)
        
            self.running_mean.mul_(1 - self.momentum).add_(self.momentum * batch_mean)
            self.running_var.mul_(1 - self.momentum).add_(self.momentum * batch_var_unbiased)
        
            x_normalized = (x - batch_mean) / torch.sqrt(batch_var + self.eps)
        else:
            x_normalized = (x - self.running_mean) / torch.sqrt(self.running_var + self.eps)

        
        return self.gamma * x_normalized + self.beta


In [31]:
def test_dropout():
    torch.manual_seed(42)
    
    x = torch.randn(4, 10)
    custom_dropout = DropoutFromScratch(p=0.5)
    pytorch_dropout = nn.Dropout(p=0.5)
    
    custom_dropout.train()
    pytorch_dropout.train()
    
    print("Input shape:", x.shape)
    print("\nTraining mode:")
    print("Custom dropout output sample:", custom_dropout(x)[0, :5])
    print("PyTorch dropout output sample:", pytorch_dropout(x)[0, :5])
    
    custom_dropout.eval()
    pytorch_dropout.eval()
    
    print("\nEval mode:")
    print("Custom dropout output:", custom_dropout(x)[0, :5])
    print("PyTorch dropout output:", pytorch_dropout(x)[0, :5])
    print("\nIn eval mode, both should equal the input:", x[0, :5])

def test_batchnorm():
    torch.manual_seed(42)
    
    batch_size, num_features = 32, 10
    x = torch.randn(batch_size, num_features)
    custom_bn = BatchNorm1dFromScratch(num_features)
    
    pytorch_bn = nn.BatchNorm1d(num_features)
    pytorch_bn.weight.data = custom_bn.gamma.data.clone()
    pytorch_bn.bias.data = custom_bn.beta.data.clone()
    
    custom_bn.train()
    pytorch_bn.train()
    
    # 🔁 Simulate multiple training steps to update running stats
    for _ in range(100):
        custom_bn(x)
        pytorch_bn(x)
    
    print("Training mode:")
    custom_out = custom_bn(x)
    pytorch_out = pytorch_bn(x)
    
    print(f"Custom output mean: {custom_out.mean():.6f}, std: {custom_out.std():.6f}")
    print(f"PyTorch output mean: {pytorch_out.mean():.6f}, std: {pytorch_out.std():.6f}")
    print(f"Outputs close? {torch.allclose(custom_out, pytorch_out, atol=1e-5)}")
    
    custom_bn.eval()
    pytorch_bn.eval()
    x_test = torch.randn(16, num_features)
    
    print("\nEval mode:")
    custom_out = custom_bn(x_test)
    pytorch_out = pytorch_bn(x_test)
    print(f"Outputs close? {torch.allclose(custom_out, pytorch_out, atol=1e-5)}")
    print(f"Max absolute diff: {(custom_out - pytorch_out).abs().max():.6e}")


In [32]:
if __name__ == "__main__":
    test_dropout()
    test_batchnorm()

Input shape: torch.Size([4, 10])

Training mode:
Custom dropout output sample: tensor([3.8538, 0.0000, 1.8014, -0.0000, 1.3568])
PyTorch dropout output sample: tensor([ 3.8538,  0.0000,  1.8014, -4.2110,  1.3568])

Eval mode:
Custom dropout output: tensor([ 1.9269,  1.4873,  0.9007, -2.1055,  0.6784])
PyTorch dropout output: tensor([ 1.9269,  1.4873,  0.9007, -2.1055,  0.6784])

In eval mode, both should equal the input: tensor([ 1.9269,  1.4873,  0.9007, -2.1055,  0.6784])
Training mode:
Custom output mean: 0.000000, std: 1.001561
PyTorch output mean: -0.000000, std: 1.001561
Outputs close? True

Eval mode:
Outputs close? True
Max absolute diff: 4.768372e-07
